In [2]:

import pickle
import numpy as np
import torch
import warnings
from functools import reduce
import matplotlib.pyplot as plt
import torch.nn as nn
import plot_nivel_tanque_new
from method import NNSolver
import os
import plot_nivel_tanque_new as plot_nivel_tanque_new
torch.xpu.is_available()
warnings.filterwarnings("ignore")
data_path = os.path.join("datasets", "dc_wss", "dc_wss_dataset_dc5_ex10")

with open(data_path, "rb") as f:
    data = pickle.load(f)
file_name = "model_2025-04-23_16-50-28_dc3_samples50_epochs5_softWeight50_td-td.pt"
file_path = os.path.join("models", file_name)
samples = int(file_name.split("samples")[1].split("_")[0])
epochs = int(file_name.split("epochs")[1].split("_")[0])
# ############################################################################

args = {
     "probType": "dc_wss",
     "hiddenSize": 200,
     "useCompl": False,
     "corrMode": "full",
     "qtySamples": samples,
     "epochs": epochs,
 }

newModel = NNSolver(data, args)
newModel.load_state_dict(torch.load(file_path, map_location=torch.device("cpu")))
newModel.eval()
print(newModel)
 ############################################################################
 # from FontinhaOptimization
 #input_data = torch.tensor([[1, 8, 12, 18, 21, 3, 3, 3, 2.5, 2.5]])
 # Random
#input_data = torch.tensor([[1, 5, 6, 7, 17, 3, 0.9, 0.9, 5, 4]])
input_data = torch.tensor([[1, 5, 6, 3, 5, 4]])
 # from resultado da FontinhaOptimization
 #input_data = torch.tensor([ [ 1.024e+00, 4.384e+00, 6.180e+00, 8.372e+00, 1.653e+01, 3.359e+00, 1.795e+00, 8.262e-01 , 4.026e-01, 3.927e+00]])
output_data = newModel(input_data)
print("Input: ", input_data)
print("Output: ", output_data)

RuntimeError: Error(s) in loading state_dict for NNSolver:
	size mismatch for net.0.weight: copying a param with shape torch.Size([200, 6]) from checkpoint, the shape in current model is torch.Size([200, 10]).
	size mismatch for net.8.weight: copying a param with shape torch.Size([6, 200]) from checkpoint, the shape in current model is torch.Size([10, 200]).
	size mismatch for net.8.bias: copying a param with shape torch.Size([6]) from checkpoint, the shape in current model is torch.Size([10]).

In [ ]:
# OPT

input_data = torch.tensor([ [ 1.024e+00, 4.384e+00, 6.180e+00, 8.372e+00, 1.653e+01, 3.359e+00, 1.795e+00, 8.262e-01 , 4.026e-01, 3.927e+00]])
valores_formatados = torch.round(input_data * 100) / 100

total_cost_opt = data.obj_fn(input_data)[0]
plot_nivel_tanque_new(5, valores_formatados[0], total_cost_opt)
print(valores_formatados)

##########################################################################

output_data_ = torch.tensor(output_data[0], dtype=torch.float32).unsqueeze(0)  # Adiciona dimensão de batch
total_cost = data.obj_fn(output_data_)[0]

# gT espera um batch, então passamos output_data_ duas vezes
#gt = data.gT(output_data_, limit=False)

# Plot
plot_nivel_tanque_new(5, output_data_[0], total_cost)






In [ ]:
example = torch.tensor([ 
                        [ 

 3.5854,  7.8650, 12.3219, 15.2064, 22.3835,  4.2786,  4.4559,  0.6188,
          3.2520,  1.3471
                
          ]
          ])
total_example = data.obj_fn(example)[0]
print(total_example)
gt_ = data.gT(example, limit=False)
print(gt_)
plot_nivel_tanque(example[0], gt_, total_example, args)





In [ ]:
a = torch.tensor([1,2,3])

b = torch.tensor([[4,5],[6,7],[8,9]])

In [ ]:
print(a.shape)
print(b.shape)



In [ ]:
c = torch.matmul(a, b)  
print(c.shape)
print(c)

In [ ]:
import data_system
import OptimAuxFunctionsV2 as op

d = data_system.data_system([5],[0])

example = torch.tensor([ 
                        [ 
1, 5, 6, 7, 17, 2, 0.9, 0.9, 0.9, 1
                
          ]
          ])


example = torch.tensor([
    
    [ 1.024e+00, 4.384e+00, 6.180e+00, 8.372e+00, 1.653e+01, 3.359e+00, 1.795e+00, 8.262e-01 , 4.026e-01, 3.927e+00]
    
    ])
    
example_ = example.detach().numpy()

tanks, timeInc, pumps = op.level_plot(example_[0], d)

print(len(tanks['tank0_h']))

print(len(timeInc['StartTime']))

In [ ]:
print(example_)

In [ ]:
plt.plot((timeInc['StartTime']/3600), pumps['pump0_s'], label='Tank 0')


In [ ]:
plt.plot((timeInc['StartTime']/3600), tanks['tank0_h'][:-1], label='Tank 0')


In [ ]:
# converte tempo para horas
time_h = timeInc['StartTime'] / 3600

# plota o nível do tanque
plt.plot(time_h, tanks['tank0_h'][:-1], label='Tank 0')

# adiciona linha horizontal em y=2
plt.axhline(y=2, color='gray', linestyle='--', linewidth=1, label='Nível 2')

# adiciona linha horizontal em y=8
plt.axhline(y=8, color='gray', linestyle='--', linewidth=1, label='Nível 8')

plt.xlabel('Tempo (h)')
plt.ylabel('Altura do tanque')
plt.legend()
plt.show()


In [ ]:
import torch
import data_system
import OptimAuxFunctionsV2 as op
import matplotlib.pyplot as plt
import numpy as np
import datetime
import os

now = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

def plot_dc_example(dc: int,
                    example,
                    total_cost: float,
                    save_plot: bool = False):
    """
    Plots pump schedule, tank level, and tariff curve.

    Parameters:
    - dc: duty-cycle (int)
    - example: control vector or Tensor
    - total_cost: total cost for annotation
    - save_plot: if True, saves the figure in 'plots/'
    """

    # --- prepare data ---
    d = data_system.data_system([dc], [0])
    ex = example.detach().cpu().numpy().reshape(-1) if torch.is_tensor(example) else example
    tanks, timeInc, pumps = op.level_plot(ex, d)
    time_h = timeInc['StartTime'] / 3600
    tank_levels = tanks['tank0_h'][:-1]

    # --- build tariff vector ---
    tar_duration = [2, 4, 1, 2, 3, 12]
    tariff_value = [0.0737, 0.06618, 0.0737, 0.10094, 0.18581, 0.10094]
    hours = np.arange(0, 24, 0.1)
    tariff_by_hour = np.zeros_like(hours)
    start = 0
    for duration, val in zip(tar_duration, tariff_value):
        end = start + duration
        mask = (hours >= start) & (hours < end)
        tariff_by_hour[mask] = val
        start = end

    # --- create figure and axes ---
    fig, ax1 = plt.subplots(figsize=(12, 6))

    # tariff curve (dashed green)
    ax1.plot(
        hours,
        tariff_by_hour,
        'g--',
        linewidth=1.5,
        label='Tariff (€ / h)'
    )
    ax1.set_xlabel('Time (h)')
    ax1.set_ylabel('Tariff (€)', color='g')
    ax1.tick_params(axis='y', labelcolor='g')
    ax1.set_xlim(0, 24)
    ax1.set_ylim(0, tariff_by_hour.max() * 1.2)

    # pump status (blue step)
    ax1.step(
        time_h,
        pumps['pump0_s'],
        where='post',
        color='b',
        label='Pump Status'
    )
    ax1.set_ylim(-0.1, 1.1)

    # --- secondary axis: tank level ---
    ax2 = ax1.twinx()
    for i in range(len(time_h) - 1):
        t0, t1 = time_h[i], time_h[i + 1]
        h0, h1 = tank_levels[i], tank_levels[i + 1]
        style = '-' if h1 >= h0 else '--'
        ax2.plot(
            [t0, t1],
            [h0, h1],
            f'r{style}',
            label='Tank Level' if i == 0 else ''
        )

    # horizontal limit lines
    ax2.axhline(2, color='gray', linestyle='--', linewidth=1, label='Min Level')
    ax2.axhline(8, color='gray', linestyle='--', linewidth=1, label='Max Level')
    ax2.set_ylabel('Tank Level', color='r')
    ax2.tick_params(axis='y', labelcolor='r')

    # title, legend, and cost annotation
    fig.suptitle(
        f'DC={dc} – Pump Schedule and Tank Level (Total Cost: €{total_cost:.2f}) — {now}'
    )
    fig.legend(
        loc='upper left',
        bbox_to_anchor=(0.85, 0.85),
        bbox_transform=ax1.transAxes
    )
    ax1.text(
        0.95,
        0.05,
        f'Total Cost: € {total_cost:.2f}',
        transform=ax1.transAxes,
        ha='right',
        va='bottom',
        fontsize=12,
        weight='bold'
    )

    plt.tight_layout()

    if save_plot:
        os.makedirs('plots', exist_ok=True)
        filename = f'plot_dc_example_dc{dc}_{now}.png'
        plt.savefig(
            os.path.join('plots', filename),
            dpi=300,
            bbox_inches='tight'
        )

    plt.show()


In [ ]:
cost_example = data.obj_fn(example)[0]
plot_dc_example(5, example, cost_example, save_plot=False)